In [308]:
import cv2
import matplotlib.pyplot as plt
import pytesseract
import re
import pandas as pd
from PIL import Image
import io
import os

# Imports the Google Cloud client library
from google.cloud import vision
from google.cloud.vision import types

os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/Users/sanghyunbyun/Desktop/rok/bin/ROKS-78e082836307.json"

plt.figure(figsize=(12, 10))
plt.style.use('dark_background')

<Figure size 864x720 with 0 Axes>

In [314]:
servers = ['1715', '1716', '1717', '1718', '1719']
n = 33
p = re.compile('[\(\)\[\]]')
p1 = re.compile('[,.]')

In [315]:
def detect_text(path):
    """Detects text in the file."""
    from google.cloud import vision
    import io
    client = vision.ImageAnnotatorClient()

    # [START vision_python_migration_text_detection]
    with io.open(path, 'rb') as image_file:
        content = image_file.read()

    image = vision.types.Image(content=content)

    response = client.text_detection(image=image)
    texts = response.text_annotations
    
    vertices = (['({},{})'.format(vertex.x, vertex.y)
                    for vertex in texts[0].bounding_poly.vertices])

    return texts[0].description

In [316]:
def chage_to_number(s_power):
    list_of_numbers = re.findall(r'\d+', s_power)
    return int(''.join(list_of_numbers))

In [410]:
for i in range(50):
    n = i+1
    initial = Image.open('./data/' + server + '/original/' + str(n) + '.png')
    width = initial.size[0]
    height = initial.size[1]

    croped_name = initial.crop((width*0.21, height*0.25, width*0.72, height*0.94))
    croped_power = initial.crop((width*0.72, height*0.25, width*0.85, height*0.94))

    croped_name.save('./data/' + server + '/name/' + str(n) + '.png')
    croped_power.save('./data/' + server + '/power/' + str(n) + '.png')

In [414]:
for i in range(50): 
    n = i+1
    initial = Image.open('./data/' + server + '/original/' + str(n) + '.png')
    width = initial.size[0]
    height = initial.size[1]

    croped = initial.crop((width*0.21, height*0.25, width*0.85, height*0.94))
    croped.save('./data/' + server + '/test/' + str(n) + '.png')

In [387]:
# for server in servers:
name = list()
allience = list()
power = list()

server = '1717'

for i in range(50):
    n = i+1
    initial = Image.open('./data/' + server + '/original/' + str(n) + '.png')
    width = initial.size[0]
    height = initial.size[1]

    croped = initial.crop((width*0.21, height*0.25, width*0.85, height*0.94))
    croped.save('./data/' + server + '/tmp.png')

    img_ori = cv2.imread('./data/' + server + '/tmp.png')

    gray = cv2.cvtColor(img_ori, cv2.COLOR_BGR2GRAY)
    structuringElement = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))

    imgTopHat = cv2.morphologyEx(gray, cv2.MORPH_TOPHAT, structuringElement)
    imgBlackHat = cv2.morphologyEx(gray, cv2.MORPH_BLACKHAT, structuringElement)

    imgGrayscalePlusTopHat = cv2.add(gray, imgTopHat)
    gray = cv2.subtract(imgGrayscalePlusTopHat, imgBlackHat)

    img_blurred = cv2.GaussianBlur(gray, ksize=(5, 5), sigmaX=0)

    ret, img_thresh = cv2.threshold(
        img_blurred, 
        170,
        255,
        cv2.THRESH_BINARY_INV
    )

    cv2.imwrite('./data/' + server + '/tmp.png',img_thresh)
    chars = detect_text('./data/' + server + '/tmp.png')

    tmp_texts = list()
    for c in chars:
        if c != '\n':
            s += c
        else:
            tmp_texts.append(s)
            s = ""

    for k in range(len(tmp_texts)):
        if len(tmp_texts) == 0:
            break

        target = tmp_texts.pop(0)
        if p.search(target):
            name.append(target)

            target = tmp_texts.pop(0)
            if p1.search(target) == None:
                allience.append(target)
                power.append(chage_to_number(tmp_texts.pop(0)))
            else:
                power.append(chage_to_number(target))
                allience.append(tmp_texts.pop(0))

        else:
            name.append(target)
            allience.append('없음')
            power.append(chage_to_number(tmp_texts.pop(0)))

df = pd.DataFrame({'name':name, 'allience':allience, 'power':power})
df.to_excel('./data/' + server + '/result.xlsx')

# plt.imshow(img_thresh, cmap='gray')

ValueError: invalid literal for int() with base 10: ''

In [416]:
name = list()
allience = list()
power = list()

for i in range(50):
    chars = detect_text('./data/1717/test/' + str(i+1) + '.png')
    tmp_texts = list()
    for c in chars:
        if c != '\n':
            s += c
        else:
            tmp_texts.append(s)
            s = ""
    
    for k in range(len(tmp_texts)):
        if len(tmp_texts) == 0:
            break

        target = tmp_texts.pop(0)
        if p.search(target):
            name.append(target)

            target = tmp_texts.pop(0)
            if p1.search(target) == None:
                allience.append(target)
                power.append(chage_to_number(tmp_texts.pop(0)))
            else:
                power.append(chage_to_number(target))
                allience.append(tmp_texts.pop(0))

        else:
            name.append(target)
            allience.append('없음')
            power.append(chage_to_number(tmp_texts.pop(0)))

In [419]:
df = pd.DataFrame({'name':name, 'allience':allience, 'power':power})
df.to_excel('./data/' + server + '/result.xlsx')

In [413]:
print(tmp_texts)
print(len(tmp_texts))

['[B717] Nelson777Neto', 'BME Strike Force', '[Cok3] 53CEOjpm', '猫の王国', '[U717] ATEM Junior', 'Undying Dreams', '[C717] #5+h', '猫の王国V', '[B717] zeldris Of B717', 'BME Strike Force', '[JGL] 40MEMO40', 'Fire & Blood']
12


In [367]:
name = list()
power = list()
allience = list()


for k in range(len(tmp_texts)):
    if len(tmp_texts) == 0:
        break

    target = tmp_texts.pop(0)
    if p.search(target):
        name.append(target)

        target = tmp_texts.pop(0)
        if p1.search(target) == None:
            allience.append(target)
            power.append(chage_to_number(tmp_texts.pop(0)))
        else:
            power.append(chage_to_number(target))
            allience.append(tmp_texts.pop(0))

    else:
        name.append(target)
        allience.append('없음')
        power.append(chage_to_number(tmp_texts.pop(0)))

In [372]:
df = pd.DataFrame({'a':name, 'v':power, 'x':allience})
df

,a,v,x
0,(UB17] SCORPIONY,38873032,Unicorn Academy B
1,[JGL] HairyHightlander,38803749,Fire & Blood
2,(17SR] Nighthawkz,38780908,SERENITY
3,(C717] REAS,38585127,猫の王国V
4,(17SR] Juice Box Hero,38047014,SERENITY
5,(8717] Brownnnnn,37782898,BME Strike Force
